In [60]:
import pandas as pd 
import numpy as np 
import torch 
from torch.utils.data import DataLoader, random_split, TensorDataset 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim 
from torch.optim import Adam 

df = pd.read_csv('heart.csv')

input = df.iloc[:, 1:-1] 

output = df.loc[:, 'output']


input = torch.Tensor(input.to_numpy())
output = torch.Tensor(output.to_numpy()).to(torch.long)
data = TensorDataset(input, output)

train_batch_size = 10        
number_rows = len(input)     
test_split = int(number_rows*0.3)  
validate_split = int(number_rows*0.2) 
train_split = number_rows - test_split - validate_split     
train_set, validate_set, test_set = random_split(data, [train_split, validate_split, test_split])

labels = {"Positive":1, "Negative":0}
train_loader = DataLoader(train_set, batch_size = train_batch_size, shuffle = True) 
validate_loader = DataLoader(validate_set, batch_size = 1) 
test_loader = DataLoader(test_set, batch_size = 1)




input_size = list(input.shape)[1]
learning_rate = 0.01 
output_size = len(labels)


class Network(nn.Module): 
   def __init__(self, input_size, output_size): 
       super(Network, self).__init__() 
        
       self.layer1 = nn.Linear(input_size, 24) 
       self.layer2 = nn.Linear(24, 24) 
       self.layer3 = nn.Linear(24, output_size) 


   def forward(self, x): 
       x1 = F.relu(self.layer1(x)) 
       x2 = F.relu(self.layer2(x1)) 
       x3 = self.layer3(x2) 
       return x3 

model = Network(input_size, output_size) 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 
print("The model will be running on", device, "device\n") 
model.to(device) 

def saveModel(): 
    path = "./PracticeModel.pth" 
    torch.save(model.state_dict(), path) 
    
    
loss_fn = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001, weight_decay=0.0001) 


def train(num_epochs): 
    best_accuracy = 0.0 
     
    print("Начало обучения") 
    for epoch in range(1, num_epochs+1): 
        running_train_loss = 0.0 
        running_accuracy = 0.0 
        running_vall_loss = 0.0 
        total = 0 
 
        # Итерация обучения
        for data in train_loader:  
            inputs, outputs = data  
            optimizer.zero_grad()          
            predicted_outputs = model(inputs)  
            train_loss = loss_fn(predicted_outputs, outputs)   
            train_loss.backward()    
            optimizer.step()       
            running_train_loss +=train_loss.item()  
 
        # Подсчет градиента потерь
        train_loss_value = running_train_loss/len(train_loader) 
 

        with torch.no_grad(): 
            model.eval() 
            for data in validate_loader: 
               inputs, outputs = data 
               predicted_outputs = model(inputs) 
               val_loss = loss_fn(predicted_outputs, outputs) 
             
               _, predicted = torch.max(predicted_outputs, 1) 
               running_vall_loss += val_loss.item()  
               total += outputs.size(0) 
               running_accuracy += (predicted == outputs).sum().item() 
 

        val_loss_value = running_vall_loss/len(validate_loader) 

        accuracy = (100 * running_accuracy / total)     

        if accuracy > best_accuracy: 
            saveModel() 
            best_accuracy = accuracy 
         

        print('Completed training batch', epoch, 'Training Loss is: %.4f' %train_loss_value, 'Validation Loss is: %.4f' %val_loss_value, 'Accuracy is %d %%' % (accuracy))

        
        
def test(): 
    model = Network(input_size, output_size) 
    path = "PracticeModel.pth" 
    model.load_state_dict(torch.load(path)) 
     
    running_accuracy = 0 
    total = 0 
 
    with torch.no_grad(): 
        for data in test_loader: 
            inputs, outputs = data 
            outputs = outputs.to(torch.float32) 
            predicted_outputs = model(inputs) 
            _, predicted = torch.max(predicted_outputs, 1) 
            total += outputs.size(0) 
            running_accuracy += (predicted == outputs).sum().item() 
 
        print('Accuracy of the model based on the test set of', test_split ,'inputs is: %d %%' % (100 * running_accuracy / total))    
        print('Ended')



def test_species(): 
    model = Network(input_size, output_size) 
    path = "PracticeModel.pth" 
    model.load_state_dict(torch.load(path)) 
     
    labels_length = len(labels) 
    labels_correct = list(0. for i in range(labels_length))  
    labels_total = list(0. for i in range(labels_length))   
  
    with torch.no_grad(): 
        for data in test_loader: 
            inputs, outputs = data 
            predicted_outputs = model(inputs) 
            _, predicted = torch.max(predicted_outputs, 1) 
             
            label_correct_running = (predicted == outputs).squeeze() 
            label = outputs[0] 
            if label_correct_running.item():  
                labels_correct[label] += 1 
            labels_total[label] += 1  
  
    label_list = list(labels.keys()) 
    for i in range(output_size): 
        print('Accuracy to predict %5s : %2d %%' % (label_list[i], 100 * labels_correct[i] / labels_total[i])) 



if __name__ == "__main__": 
    #num_epochs = 1000
    #train(num_epochs) 
    #print('Обучение закончено\n') 
    #test() 
    test_species()



The model will be running on cpu device

Accuracy to predict Positive : 85 %
Accuracy to predict Negative : 77 %


In [6]:
import pandas as pd 
import numpy as np 
import torch 
from torch.utils.data import DataLoader, random_split, TensorDataset 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim 
from torch.optim import Adam 

df = pd.read_csv('heart.csv')

input = df.iloc[:, 1:-1] 

output = df.loc[:, 'output']


#input = torch.Tensor(input.to_numpy())
#output = torch.Tensor(output.to_numpy()).to(torch.long)
#data = TensorDataset(input, output)

response = f'Входные данные:\n {input.head()}\n\nВыходные данные:\n{output.head()}'
response

'Входные данные:\n    sex  cp  trtbps  chol  fbs  restecg  thalachh  exng  oldpeak  slp  caa  \\\n0    1   3     145   233    1        0       150     0      2.3    0    0   \n1    1   2     130   250    0        1       187     0      3.5    0    0   \n2    0   1     130   204    0        0       172     0      1.4    2    0   \n3    1   1     120   236    0        1       178     0      0.8    2    0   \n4    0   0     120   354    0        1       163     1      0.6    2    0   \n\n   thall  \n0      1  \n1      2  \n2      2  \n3      2  \n4      2  \n\nВыходные данные:\n0    1\n1    1\n2    1\n3    1\n4    1\nName: output, dtype: int64'

torch.int64

In [20]:
import pandas as pd 
import numpy as np 
import torch 
from torch.utils.data import DataLoader, random_split, TensorDataset 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim 
from torch.optim import Adam 

df = pd.read_csv('heart.csv')

input = df.iloc[:, 1:-1] 

output = df.loc[:, 'output']


input = torch.Tensor(input.to_numpy())
output = torch.Tensor(output.to_numpy()).to(torch.long)
data = TensorDataset(input, output)

train_batch_size = 30        
number_rows = len(input)     
test_split = int(number_rows*0.3)  
validate_split = int(number_rows*0.2) 
train_split = number_rows - test_split - validate_split     
train_set, validate_set, test_set = random_split(data, [train_split, validate_split, test_split])

labels = {"Positive":1, "Negative":0}
train_loader = DataLoader(train_set, batch_size = 1) 
validate_loader = DataLoader(validate_set, batch_size = 1) 
test_loader = DataLoader(test_set, batch_size = 1)

print(f'Данных на обучение: {len(train_loader)}\nДанных на тестирование: {len(test_loader)}\nДанных на проверку: {len(validate_loader)}')



Данных на обучение: 153
Данных на тестирование: 90
Данных на проверку: 60
